In [ ]:
!pip install --quiet tensorflow-federated
import numpy as np
import random
import cv2
import pandas as pd
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import collections
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
import tensorflow_federated as tff

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.2/365.2 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.0 MB/s eta 0:0

In [ ]:
data_train = pd.read_csv('KDDTrain+.txt')
data_test = pd.read_csv('KDDTest+.txt')
data_train.columns

Index(['0', 'tcp', 'ftp_data', 'SF', '491', '0.1', '0.2', '0.3', '0.4', '0.5',
       '0.6', '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14',
       '0.15', '0.16', '0.18', '2', '2.1', '0.00', '0.00.1', '0.00.2',
       '0.00.3', '1.00', '0.00.4', '0.00.5', '150', '25', '0.17', '0.03',
       '0.17.1', '0.00.6', '0.00.7', '0.00.8', '0.05', '0.00.9', 'normal',
       '20'],
      dtype='object')

In [ ]:
data_full = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
data_full = pd.DataFrame(data_full)
data_full.columns = data_full.columns.str.replace(" ", "")

In [ ]:
data_test = tf.data.Dataset.from_tensor_slices(np.array(data_full))


In [ ]:
mapping_dict = {'BENIGN':0, 'DDOS':1}
data_full['Label'] = data_full['Label'].map(mapping_dict)
labels = data_full['Label']
data_full.drop(['Label'], axis = 1, inplace = True)
x_train, x_test, y_train, y_test = train_test_split(data_full, labels, test_size = 0.2, random_state = 10)

In [ ]:
x_train

#Create nodes for separation of data

In [ ]:
def create_nodes(data, labels, num_nodes: int):
  init = 'Node'
  node_names = [init+str(i+1) for i in range(num_nodes)]
  data = np.array(data)

  data = list(zip(data, labels))
  random.shuffle(data)

  shard_size = len(data)//num_nodes

  shards = [data[i:i+shard_size] for i in range(0, shard_size*num_nodes, shard_size)]
  print(len(shards))
  assert(len(shards) == num_nodes)

  return {node_names[i] : shards[i] for i in range(len(node_names))}

nodes = create_nodes(x_train, y_train, 50)

def batch_data(shard):
  data, label = zip(*shard)
  dataset = tf.data.Dataset.from_tensor_slices((np.array(data), np.array(label)))
  return dataset.shuffle(len(label)).batch(32)



50


In [ ]:
#process and batch the training data for each client
nodes_batched = dict()
for (node_name, data) in nodes.items():

  print(batch_data(data))
  nodes_batched[node_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(len(y_test))

In [ ]:
class MLP:
    @staticmethod
    def build(shape):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(1))
        model.add(Activation("softmax"))
        return model

lr = 0.01
comms_round = 100
loss='binary_crossentropy'
metrics = ['accuracy']

In [ ]:

def weight_scaling_factor(node_train, node_name):
    node_names = list(node_train.keys())
    #get the bs
    bs = list(node_train[node_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(node_train[node_name]).numpy() for node_name in node_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(node_train[node_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.BinaryCrossentropy()
    logits = model.predict(X_test)
    print(logits)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:

#initialize global model
smlp_global = MLP()
global_model = smlp_global.build(78)

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    node_names= list(nodes_batched.keys())
    random.shuffle(node_names)

    #loop through each client and create new local model
    for node in node_names:
        smlp_local = MLP()
        local_model = smlp_local.build(78)
        local_model.compile(loss=loss,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(nodes_batched[node], epochs=3)

        #scale the model weights and add to list
        scaling_factor = weight_scaling_factor(nodes_batched, node)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(x_test, y_test) in test_batched:
        global_acc, global_loss = test_model(x_test, y_test, global_model, comm_round)

In [ ]:
for(x_test, y_test) in test_batched:
        global_acc, global_loss = test_model(x_test, y_test, global_model, comm_round)

#Baseline Model

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = MLP()
SGD_model = smlp_SGD.build(x_train.shape[1])

SGD_model.compile(loss=loss,
              optimizer=tf.keras.optimizers.Adam,
              metrics=metrics)

# fit the SGD training data to model
fitted_model = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, fitted_model, 1)

#Federated Learning with Differential Privacy

In [ ]:
def get_emnist_dataset():
  emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(
      only_digits=True)

  def element_fn(element):
    return collections.OrderedDict(
        x=tf.expand_dims(element['pixels'], -1), y=element['label'])

  def preprocess_train_dataset(dataset):
    # Use buffer_size same as the maximum client dataset size,
    # 418 for Federated EMNIST
    return (dataset.map(element_fn)
                   .shuffle(buffer_size=418)
                   .repeat(1)
                   .batch(32, drop_remainder=False))

  def preprocess_test_dataset(dataset):
    return dataset.map(element_fn).batch(128, drop_remainder=False)

  emnist_train = emnist_train.preprocess(preprocess_train_dataset)
  emnist_test = preprocess_test_dataset(
      emnist_test.create_tf_dataset_from_all_clients())
  return emnist_train, emnist_test

train_data, test_data = get_emnist_dataset()

In [ ]:
train_data.shape

AttributeError: ignored